<a href="https://colab.research.google.com/github/umaabhyankar/AQI-Predictor/blob/main/feature_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U hopsworks --quiet

In [2]:
!pip install geopy folium streamlit-folium --q

In [55]:
import datetime
import time
import requests
import json

import numpy as np
import pandas as pd
import folium
import os

from feature_funcs import *

import warnings
warnings.filterwarnings("ignore")

# Feature Engineering

In [62]:
df = pd.read_csv('data/solapur_india_air_quality.csv')

In [63]:
df

,date,pm25,pm10,o3,no2,so2,co
0,2023/7/1,116,88,18,18,11,
1,2023/7/2,95,85,18,18,11,
2,2023/7/3,92,78,20,18,11,
3,2023/7/4,85,75,20,18,11,
4,2023/7/5,81,75,20,17,12,
...,...,...,...,...,...,...,...
2266,2021/7/18,,,,3,,
2267,2021/8/29,,,,3,,
2268,2021/9/30,,,,2,,
2269,2021/6/20,,,,2,,


In [64]:
df['date'] = pd.to_datetime(df['date'])

In [65]:
df.date.max()

Timestamp('2023-07-07 00:00:00')

In [66]:
df[' pm25'] = [int(i) if i.isdigit() else 0 for i in [i[1:] for i in df[' pm25']]]
df[' pm10'] = [int(i) if i.isdigit() else 0 for i in [i[1:] for i in df[' pm10']]]
df[' o3'] = [int(i) if i.isdigit() else 0 for i in [i[1:] for i in df[' o3']]]
df[' no2'] = [int(i) if i.isdigit() else 0 for i in [i[1:] for i in df[' no2']]]
df[' so2'] = [int(i) if i.isdigit() else 0 for i in [i[1:] for i in df[' so2']]]
df[' co'] = [int(i) if i.isdigit() else 0 for i in [i[1:] for i in df[' co']]]

In [67]:
df=df.loc[(df[' pm25'] != 0) | (df[' pm10'] != 0)]

In [68]:
df.sort_values(by='date', inplace=True)

In [69]:
df.reset_index(drop=True, inplace=True)

In [70]:
df['n_count'] = np.count_nonzero(df[[' pm25', ' pm10', ' o3', ' no2', ' so2', ' co']], axis=1)

In [71]:
df = df.loc[df['n_count']>=3]

In [72]:
df.rename(columns = {' pm25':'pm25', ' pm10':'pm10', ' o3':'o3', ' no2':'no2', ' so2':'so2', ' co':'co'},inplace=True)

In [73]:
df.columns

Index(['date', 'pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'n_count'], dtype='object')

In [74]:
response_API = requests.get('https://api.waqi.info/feed/solapur/?token=6e62f66525787498e49dd5df9d7a3993ff183759')
#print(response_API.status_code)

In [75]:
data = response_API.text

In [76]:
df_data = json.loads(data)

In [78]:
df_forecast = pd.DataFrame(columns = ['date', 'average', 'pollutant'])
for pollutant, values in df_data['data']['forecast']['daily'].items():
  # print(pollutant)
  # print(values)
  df_forecast_x = pd.DataFrame()
  # for val in values:
  #   print(val['avg'])
  df_forecast_x['date'] = [i['day'] for i in values]
  df_forecast_x['average'] = [i['avg'] for i in values]
  df_forecast_x['pollutant'] = [pollutant] * len(values)
  # print(df_forecast_x)
  df_forecast = df_forecast.append(df_forecast_x)
  # pd.concat([df_forecast,df_forecast_x])
  # print(df_forecast)

In [79]:
table = pd.pivot_table(df_forecast, values ='average', index =['date'],
                         columns =['pollutant'], aggfunc = np.sum)

In [80]:
table.reset_index(inplace = True)

In [81]:
table = table[['date','pm25','pm10','o3']]

In [82]:
table['no2'] = [0]*len(table)
table['so2'] = [0]*len(table)
table['co'] = [0]*len(table)

In [83]:
table['date'] = pd.to_datetime(table['date'])

In [84]:
table.fillna(0, inplace=True)

In [85]:
df = df.append(table)

In [86]:
pm10_si_list = []
pm25_si_list = []
so2_si_list = []
no2_si_list = []
co_si_list = []
o3_si_list = []

for i in df['pm10']:
    pm10_si_list.append(pm10_sub_index(i))

for i in df['pm25']:
    pm25_si_list.append(pm25_sub_index(i))

for i in df['o3']:
    o3_si_list.append(o3_sub_index(i))

for i in df['no2']:
    no2_si_list.append(no2_sub_index(i))

for i in df['so2']:
    so2_si_list.append(so2_sub_index(i))

for i in df['co']:
    co_si_list.append(co_sub_index(i))

In [88]:
df['pm10_si'] = pm10_si_list
df['pm25_si'] = pm25_si_list
df['o3_si'] = o3_si_list
df['no2_si'] = no2_si_list
df['so2_si'] = so2_si_list
df['co_si'] = co_si_list

In [90]:
df['AQI'] = df[['pm10_si', 'pm25_si', 'o3_si', 'no2_si', 'so2_si', 'co_si']].max(axis = 1)

In [91]:
df['no2'] = df['no2'].astype('float')
df['so2'] = df['so2'].astype('float')
df['co'] = df['co'].astype('float')

,date,pm25,pm10,o3,no2,so2,co,n_count,pm10_si,pm25_si,o3_si,no2_si,so2_si,co_si,AQI
0,2016-01-03,0.0,126.0,18.0,18,8,22,5.0,117.333333,0.000000,18.0,22.50,10.0,329.411765,329.411765
1,2016-01-04,145.0,121.0,16.0,17,8,27,6.0,114.000000,319.230769,16.0,21.25,10.0,358.823529,358.823529
2,2016-01-05,147.0,103.0,15.0,16,8,30,6.0,102.000000,320.769231,15.0,20.00,10.0,376.470588,376.470588
3,2016-01-06,151.0,115.0,16.0,16,8,26,6.0,110.000000,323.846154,16.0,20.00,10.0,352.941176,352.941176
4,2016-01-07,147.0,103.0,16.0,16,8,23,6.0,102.000000,320.769231,16.0,20.00,10.0,335.294118,335.294118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2023-07-12,121.0,58.0,6.0,0,0,0,NaN,58.000000,300.769231,6.0,0.00,0.0,0.000000,300.769231
5,2023-07-13,97.0,50.0,6.0,0,0,0,NaN,50.000000,223.333333,6.0,0.00,0.0,0.000000,223.333333
6,2023-07-14,70.0,31.0,5.0,0,0,0,NaN,31.000000,133.333333,5.0,0.00,0.0,0.000000,133.333333
7,2023-07-15,80.0,45.0,4.0,0,0,0,NaN,45.000000,166.666667,4.0,0.00,0.0,0.000000,166.666667


In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

In [ ]:
!pip install --upgrade jinja2==3.0.3

In [ ]:
!pip uninstall jinja2

In [104]:
AQI_feature_group = fs.get_or_create_feature_group(
    name='air_quality',
    description='Parameters for AQI',
    version=1,
    primary_key=['date'],
    online_enabled=False,

)

In [ ]:
AQI_feature_group.insert(df, write_options={"wait_for_job": False})